In [ ]:
import pandas as pd
import numpy as np

aisle = pd.read_csv('C:/Users/lyn02/Desktop/CMPE255 Project/Raw Data/aisles.csv')
department = pd.read_csv('C:/Users/lyn02/Desktop/CMPE255 Project/Raw Data/departments.csv')
order_products_prior = pd.read_csv('C:/Users/lyn02/Desktop/CMPE255 Project/Raw Data/order_products__prior.csv')
order_products_train = pd.read_csv('C:/Users/lyn02/Desktop/CMPE255 Project/Raw Data/order_products__train.csv')
orders = pd.read_csv('C:/Users/lyn02/Desktop/CMPE255 Project/Raw Data/orders.csv')
products = pd.read_csv('C:/Users/lyn02/Desktop/CMPE255 Project/Raw Data/products.csv')


In [ ]:
# Take out orderID and productID from both train and prior table and merge them into one order_products_id table
order_products_id1 = order_products_train[['order_id','product_id']]
order_products_id2 = order_products_prior[['order_id','product_id']]
order_products_id = order_products_id1.append(order_products_id2, ignore_index=True)
order_products_id

In [ ]:
# take out orderID and userID from orders table
order_user_id = orders[['order_id','user_id']]
order_user_id

In [ ]:
# to see how many orders each user has altogether
user_order_count = order_user_id[['user_id','order_id']]
user_order_count['order_id'] = order_user_id['user_id']
user_order_count = user_order_count.groupby(user_order_count.columns.tolist()).size().reset_index().rename(columns={0:'count'})
user_order_count = user_order_count[['user_id','count']]
user_order_count

In [ ]:
#combine order_products and products table to see what aisle each product belongs for each order
order_product_aisle = order_products_id.merge(products,on='product_id')
order_product_aisle

In [ ]:
#from order_product_aisle table, only select orderID and aisleID to aggregate some data since we only deal with 
#aisle level granularity
#In addition, count of products from each aisle for each order is kept
order_aisle_id = order_product_aisle[['order_id','aisle_id']]
order_aisle_id
order_aisle_count = order_aisle_id.groupby(order_aisle_id.columns.tolist()).size().reset_index().rename(columns={0:'count'})
order_aisle_count

In [ ]:
#join order_user_id table and order_aisle_count table to see which order belongs to which user
order_user_aisle = order_aisle_count.merge(order_user_id, on='order_id')
order_user_aisle

In [ ]:
#from order_user_aisle table, take out orderID
user_aisle_id = order_user_aisle[['user_id','aisle_id','count']]
user_aisle_id

In [ ]:
# continue from previos step, we maintain count of orders for each aisle belonging to each user
user_aisle_count = user_aisle_id.groupby(user_aisle_id.columns.tolist()).size().reset_index().rename(columns={0:'rcount'})
user_aisle_count

In [ ]:
#continue from previous step, we multiple two count column to get how many products each user bought from each aisle altogether
user_aisle_count['count'] = user_aisle_count['count']*user_aisle_count['rcount']

In [ ]:
user_aisle_count = user_aisle_count[['user_id','aisle_id','count']]
user_aisle_count

In [ ]:
# user_order_count left join user_aisle_count
#count_x = product count
#count_y = order count
user_aisle_order_count = user_aisle_count.merge(user_order_count, how='left',on='user_id')
user_aisle_order_count


In [ ]:
# divide product count by order count to get for aisle, how many products on average each customer would buy per order
user_aisle_order_count['avg'] = user_aisle_order_count['count_x']/user_aisle_order_count['count_y']
user_aisle_avgCount = user_aisle_order_count[['user_id','aisle_id','avg']]
user_aisle_avgCount

In [ ]:
#pivot table
table = pd.pivot_table(user_aisle_avgCount, values='avg', index=['user_id'],columns=['aisle_id'], aggfunc=np.sum, fill_value=0)

In [ ]:
table